<a href="https://colab.research.google.com/github/heloisatambara/computacao-e-simulacao/blob/main/naoSeiOqToFzndAinda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from scipy.stats import chi2
from scipy.optimize import minimize_scalar

import numpy as np
#import matplotlib.pyplot as plt
from datetime import datetime

import math 

In [3]:
def listOfXvectors(): # cria a lista dos vetores x a serem testados, exemplificados no artigo
    Xvectors = []
    for x3 in range(2, 19):
        Xvectors.append([1, 19-x3, x3])
    for x3 in range(0,11):
        Xvectors.append([5, 15-x3, x3])
    for x3 in range(0,8):
        Xvectors.append([9, 11-x3, x3])

    return Xvectors



def theta(theta1,theta3):
    # funcao potencial
    theta2 = 1 - theta1 - theta3
    if theta1 <0 or theta2<0 or theta3<0:
        return 0
    y = ((theta1**(x1-1))*(theta2**(x2-1))*(theta3**(x3-1)))
    return y
    

def alpha(i,j):
    # alpha de aceitacao utilizado
    if theta(i[0],i[1]) == 0: return 1
    return theta(j[0],j[1])/theta(i[0],i[1])


    
###############################################################################
    
    
def MCMC(aquecimento,tamanho_da_cadeia,saltos,mean,cov,ponto_inicial):
    #gera uma cadeia a partir do n do aquecimento, n do tamanho da cadeia e
    # tamanho dos saltos, matriz de covariancia e ponto inicial
    # funciona apenas para dimensao dois
    
    
    # total de pontos a serem gerados
    n = aquecimento+(tamanho_da_cadeia*saltos)
    
    
    # gera os valores da normal e uniforme anteriormente por vetorizacao
    normal = np.random.multivariate_normal(mean,cov,n)
    uniforme = np.random.uniform(0,1,n)
    
    #contagem dos pontos gerados para pegar o k-esimo candidato
    k = 0
    
    
    #cria a base da cadeia, apenas para facilitar o codigo
    cadeia = [0]*(tamanho_da_cadeia*saltos)

    
    
    
                        #### Aquecimento ####   
    # funcionamento igual ao topico MCMC, sem salvar a cadeia
    # Salvando apenas o ultimo membro
    cadeia_fria = ponto_inicial 
    for i in range(aquecimento):   
        atual = cadeia_fria
        proximo = (0,0)
        caminho = normal[k]
        proximo = (atual[0] + caminho[0] , atual[1] + caminho[1])
        if uniforme[k] < alpha(atual , proximo):
            cadeia_fria = proximo
        k += 1
        
        
        
    # calculo da taxa de aceitacao
    taxa_num = 0
    
            
            
                        #### MCMC ####   
        #######################################################################
    cadeia[0] = cadeia_fria        #A cadeia esquentada recebe o ultimo 
                                     #termo da fria
        #######################################################################
    for i in range((tamanho_da_cadeia*saltos)-1):     
        atual = cadeia[i]    # Posicao atual da cadeia
        #######################################################################
        # Calculo do proximo candidato
        proximo = (0,0)   
        caminho = normal[k]
        proximo = (atual[0] + caminho[0] , atual[1] + caminho[1])
        #######################################################################
                                                   # Decide se a nova posicao é
        if uniforme[k] < alpha(atual , proximo):    # aceita, movendo a 
            cadeia[i+1] = proximo                            # cadeia caso seja 
            
            taxa_num += 1 #calcula a taxa de aceitacao
            
        else:
            #repete o ultimo termo caso n seja aceito
            cadeia[i+1] = atual
            
        k += 1
        #######################################################################  
        
    #taxa de aceitacao
    taxa = taxa_num/len(cadeia)
        
    
    
    if saltos ==1:
        return cadeia,taxa
    
    # elimina parte da amostra para diminuir a correlacao
    cadeia_final = [0]*tamanho_da_cadeia
    for i in range(tamanho_da_cadeia):
        cadeia_final[i] = cadeia[saltos*i]
    return cadeia_final,taxa # a cadeia final é s?

def u(c,v):
      # calculo de u a partir da cadeia e de um vetor v de cortes, retorna vetor
      soma = [0]*len(v)
      for j in range(len(v)):
          # aqui q entra a ctt de integracao
          k = v[j]/(math.gamma(x1+x2+x3)/(math.gamma(x1)*math.gamma(x2)*math.gamma(x3)))
          for i in c:
              if theta(i[0],i[1]) > k:
                  soma[j] += 1
      somas = np.array(soma)
      return 1 - (somas/len(c) )



    

In [65]:
num = 100000  #numero de pontos
x = [1,2,3]  #vetor x
y = [4,6,4]  #vetor y
v = [0,1,0.5,7,15]  # valores a serem calculados


# media da dist normal utilizada
mean = [0,0]
# definificao da matriz de covariancia utilizada
# dimensao 2 pois o terceiro valor eh um menos os outros dois
sigma = 0.02
cov = np.array([[sigma,0],[0,sigma]])
#Ponto inicial da cadeia
ponto_inicial = (0,0)
saltos = 10
n_cadeia_fria = 10000
###############################################################################

x1,x2,x3 =  [x[0]+y[0],x[1]+y[1],x[2]+y[2]]  # vetor x+y

#calcula o tempo
now1 = datetime.now()

#gera a cadeia
c,taxa = MCMC(n_cadeia_fria,num,saltos,mean,cov,ponto_inicial)   #           

#calcula os u(v)
lista = u(c,v)

In [5]:
    
string = ""
for i in range(len(v)):
  string += f"U({v[i]}) = {lista[i]}\n"
print("As estimativas foram: ")
print(string)

print(f"A taxa de aceitacao foi de {taxa}%")
now2 = datetime.now()
print(f"Programa demorou {now2-now1} para rodar.\n")
print(f"Foram gerados {n_cadeia_fria+(num*saltos)} pontos sendo",
      f"{n_cadeia_fria} o tamanho da cadeia fria e {num*saltos} a",
      f"quantia gerada para se ter uma cadeia de tamanho {num} com",
      f"{saltos} saltos.\n")
print(f"A cadeia se iniciou no ponto {ponto_inicial} e variou a partir",
      "de uma normal multivariada com a seguinte distribuicao:")

string = f"  / |{mean[0]}| |{str(cov[0])[1:-1]}| \\\n"
string+= f"N \ |{mean[1]}|,|{str(cov[1])[1:-1]}| /"
print(string)

As estimativas foram: 
U(0) = 0.0
U(1) = 0.04037999999999997
U(0.5) = 0.018179999999999974
U(7) = 0.37538000000000005
U(15) = 0.88988

A taxa de aceitacao foi de 0.388258%
Programa demorou 0:00:23.944414 para rodar.

Foram gerados 1010000 pontos sendo 10000 o tamanho da cadeia fria e 1000000 a quantia gerada para se ter uma cadeia de tamanho 100000 com 10 saltos.

A cadeia se iniciou no ponto (0, 0) e variou a partir de uma normal multivariada com a seguinte distribuicao:
  / |0| |0.02 0.  | \
N \ |0|,|0.   0.02| /


In [66]:
def getThetaVector(theta):
  thetaVector = [theta, 1 - theta - (1-theta**(1/2))**2, (1-theta**(1/2))**2]
  return thetaVector
  
def fdps(theta): # função densidade probabilidade - função S a ser maximizada 
    thetaVector = [theta, 1 - theta - (1-theta**(1/2))**2, (1-theta**(1/2))**2]
    return -(thetaVector[0]**x[0]*thetaVector[1]**x[1]*thetaVector[2]**x[2]) # negativo pois a função minimiza seu valor

def listOfXvectors(): # cria a lista dos vetores x a serem testados, mostrados no artigo
    Xvectors = []
    for x3 in range(2, 19):
        Xvectors.append([1, 19-x3, x3])
    for x3 in range(0,11):
        Xvectors.append([5, 15-x3, x3])
    for x3 in range(0,8):
        Xvectors.append([9, 11-x3, x3])
    return Xvectors

def sev(t,h,ev): # calcula o evalor normalizado
    y = 1- QQ(t,h,1-ev)
    return y

t = 3 #dim(theta)
h = 2 #dim(H)

def QQ(t,h,z): 
    fz = chi2.cdf( chi2.ppf(z,t), t*h)
    return fz



In [70]:
x = [1,17,2] # teste
a = minimize_scalar(fdps)
print(a)
print(a.get('x'))
print(a.get('fun'))
print(fdps(a.get('x')))

     fun: -1.253238940258808e-07
    nfev: 20
     nit: 16
 success: True
       x: 0.225625004470036
0.225625004470036
-1.253238940258808e-07
-1.253238940258808e-07


In [79]:
Y = [1,1,1] # priori
Xvectors = listOfXvectors() # gera todos os X na tabela
for X in Xvectors: # para cada vetor X: 
    x1, x2, x3 = X[0]+1, X[1]+1, X[2]+1  # 
    maximizando = minimize_scalar(fdps)
    s_estrela = -maximizando.get('fun') # acha o valor máximo de s
    ev = u(c, [s_estrela]) # acha W(s*) = ev(H|x) - e valor
    vetor_theta = getThetaVector(maximizando.get('x')) # acha o vetor theta
    Sev = sev(t,h,ev) # calcula e valor padronizado
    if Sev <= .05: H = "Rejeita H0" # decide se aceita ou rejeita H0
    else: H = "Aceita H0"

    print(f" x1={x1-1} , x3={x3-1}, Y= {Y}, H={H},  θ*= {vetor_theta}, ev(H|X)={format(ev[0],'.3f')} e sev(H|X)={format(Sev[0],'.3f')}")

 x1=1 , x3=2 , Y= [1, 1, 1], H=Aceita H0,  θ*= [0.225625004470036, 0.4987500004705301, 0.27562499505943394], ev(H|X)=0.023 e sev(H|X)=0.144
 x1=1 , x3=3 , Y= [1, 1, 1], H=Aceita H0,  θ*= [0.225625004470036, 0.4987500004705301, 0.27562499505943394], ev(H|X)=0.009 e sev(H|X)=0.069
 x1=1 , x3=4 , Y= [1, 1, 1], H=Rejeita H0,  θ*= [0.225625004470036, 0.4987500004705301, 0.27562499505943394], ev(H|X)=0.003 e sev(H|X)=0.031
 x1=1 , x3=5 , Y= [1, 1, 1], H=Rejeita H0,  θ*= [0.225625004470036, 0.4987500004705301, 0.27562499505943394], ev(H|X)=0.001 e sev(H|X)=0.013
 x1=1 , x3=6 , Y= [1, 1, 1], H=Rejeita H0,  θ*= [0.225625004470036, 0.4987500004705301, 0.27562499505943394], ev(H|X)=0.000 e sev(H|X)=0.006
 x1=1 , x3=7 , Y= [1, 1, 1], H=Rejeita H0,  θ*= [0.225625004470036, 0.4987500004705301, 0.27562499505943394], ev(H|X)=0.000 e sev(H|X)=0.002
 x1=1 , x3=8 , Y= [1, 1, 1], H=Rejeita H0,  θ*= [0.225625004470036, 0.4987500004705301, 0.27562499505943394], ev(H|X)=0.000 e sev(H|X)=0.001
 x1=1 , x3=9 , 

In [75]:
Y = [0,0,0] # priori
Xvectors = listOfXvectors() # gera todos os X na tabela
for X in Xvectors: # para cada vetor X:
    x1, x2, x3 = X[0], X[1], X[2]
    maximizando = minimize_scalar(fdps)
    s_estrela = -maximizando.get('fun') # acha o valor máximo de s
    vetor_theta = getThetaVector(maximizando.get('x'))
    try:
      ev = u(c, [s_estrela]) # acha W(s*) = ev(H|x)
      Sev = sev(t,h,ev)
      if Sev <= .05: H = "Rejeita H0"
      else: H = "Aceita H0"
      print(f"x1={x1}, x3={x3}, Y= {Y}, H={H},  θ*= {vetor_theta}, ev(H|X)={ev} e sev(H|X)={Sev}")
    except:
      print(f"x1={x1}, x3={x3}, Y= {Y} - não calculados, erro de domínio")

x1=1, x3=2, Y= [0, 0, 0], H=Aceita H0,  θ*= [0.225625004470036, 0.4987500004705301, 0.27562499505943394], ev(H|X)=[0.04665] e sev(H|X)=[0.24035948]
x1=1, x3=3, Y= [0, 0, 0], H=Aceita H0,  θ*= [0.225625004470036, 0.4987500004705301, 0.27562499505943394], ev(H|X)=[0.01627] e sev(H|X)=[0.11303505]
x1=1, x3=4, Y= [0, 0, 0], H=Aceita H0,  θ*= [0.225625004470036, 0.4987500004705301, 0.27562499505943394], ev(H|X)=[0.00565] e sev(H|X)=[0.05029046]
x1=1, x3=5, Y= [0, 0, 0], H=Rejeita H0,  θ*= [0.225625004470036, 0.4987500004705301, 0.27562499505943394], ev(H|X)=[0.00179] e sev(H|X)=[0.02001379]
x1=1, x3=6, Y= [0, 0, 0], H=Rejeita H0,  θ*= [0.225625004470036, 0.4987500004705301, 0.27562499505943394], ev(H|X)=[0.00049] e sev(H|X)=[0.00682642]
x1=1, x3=7, Y= [0, 0, 0], H=Rejeita H0,  θ*= [0.225625004470036, 0.4987500004705301, 0.27562499505943394], ev(H|X)=[0.00026] e sev(H|X)=[0.00398907]
x1=1, x3=8, Y= [0, 0, 0], H=Rejeita H0,  θ*= [0.225625004470036, 0.4987500004705301, 0.27562499505943394], ev